In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import nn
import pandas as pd
import numpy as np
import pickle

In [3]:
with open('x_train.pkl', 'rb') as f:
    x_train = pickle.load(f)

with open('y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('x_valid.pkl', 'rb') as f:
    x_valid = pickle.load(f)

with open('y_valid.pkl', 'rb') as f:
    y_valid = pickle.load(f)

with open('x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)

with open('y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

In [4]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)
print(x_test.shape,y_valid.shape, y_test.shape)

(24500,) (1121,) (24500,) (1121,)
(2567,) (1121,) (2567,)


In [5]:
class TransformerBlock(layers.Layer):  #공통적인 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(embedding_dim=embed_dim, num_heads=num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        #print('어텐션 후',attn_output.shape)
        attn_output = self.dropout1(attn_output, training=training)
        #print('드롭아웃',attn_output.shape)
        out1 = self.layernorm1(inputs + attn_output)
        #print('레이어놈',out1.shape)
        ffn_output = self.ffn(out1)
        #print('ffn후',ffn_output.shape)
        ffn_output = self.dropout2(ffn_output, training=training)
        print("트랜스포머")
        return self.layernorm2(out1 + ffn_output)
class MultiHeadAttention(tf.keras.layers.Layer): #공통적인 트랜스포머 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=4):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

In [6]:
from tensorflow.keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score


In [7]:
class eda_TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, embed_dimm):
        super(eda_TokenAndPositionEmbedding, self).__init__()
        self.temp_emb = tf.keras.layers.Embedding(input_dim=max_len, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = 80
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.temp_emb(x)
        return x + positions

In [8]:
embed_dim = 32 # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen=50
training=1

eda_input = layers.Input(shape=(80,)) #
eda_embedding_layer = eda_TokenAndPositionEmbedding(80, embed_dim)
eda_embedding = eda_embedding_layer(eda_input)  #

transformer_block = TransformerBlock(32, num_heads, ff_dim) 

output = transformer_block(eda_embedding)
output = transformer_block(output)

x = layers.GlobalAveragePooling1D()(output)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="sigmoid")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(7, activation="softmax")(x)  
print(outputs.shape)

model = keras.Model(inputs=eda_input, outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

트랜스포머
트랜스포머
(None, 7)


In [9]:
history = model.fit(
    [np.array(x_train.to_list())], y_train, batch_size=16, epochs=5, validation_data=([np.array(x_valid.to_list())], y_valid)
)

Epoch 1/5
트랜스포머
트랜스포머
트랜스포머
트랜스포머
1528/1532 [============================>.] - ETA: 0s - loss: 1.7948 - accuracy: 0.2597트랜스포머
트랜스포머
1532/1532 [==============================] - 25s 15ms/step - loss: 1.7946 - accuracy: 0.2599 - val_loss: 1.4189 - val_accuracy: 0.4835
Epoch 2/5
1532/1532 [==============================] - 23s 15ms/step - loss: 1.5687 - accuracy: 0.3723 - val_loss: 1.5095 - val_accuracy: 0.4166
Epoch 3/5
1532/1532 [==============================] - 22s 14ms/step - loss: 1.4968 - accuracy: 0.3996 - val_loss: 1.5304 - val_accuracy: 0.4442
Epoch 4/5
1532/1532 [==============================] - 23s 15ms/step - loss: 1.4509 - accuracy: 0.4129 - val_loss: 1.3781 - val_accuracy: 0.4585
Epoch 5/5
1532/1532 [==============================] - 22s 15ms/step - loss: 1.4064 - accuracy: 0.4310 - val_loss: 1.4363 - val_accuracy: 0.4389


In [10]:
x_test_np = np.array(x_test)

In [11]:
list_col = x_test
list_col_array = np.array(list_col.tolist())
new_df = pd.DataFrame(list_col_array, columns=[f'col{i+1}' for i in range(list_col_array.shape[1])])

In [12]:
new_df

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80
0,33.07,33.09,33.09,33.09,33.09,33.09,33.09,33.09,33.09,33.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33.31,33.31,33.31,33.31,33.31,33.31,33.31,33.31,33.31,33.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33.07,33.07,33.05,33.05,33.05,33.05,33.03,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,33.27,33.27,33.25,33.25,33.25,33.25,33.27,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33.03,33.03,33.03,33.03,33.03,33.03,33.03,33.05,33.05,33.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,35.07,35.07,35.07,35.07,35.05,35.05,35.05,35.05,35.05,35.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2563,35.05,35.05,35.07,35.07,35.07,35.07,35.07,35.07,35.07,35.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2564,35.05,35.05,35.05,35.05,35.05,35.05,35.05,35.05,35.05,35.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2565,35.05,35.07,35.07,35.07,35.07,35.07,35.07,35.07,35.07,35.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
x_test_np = np.array(new_df)
x_test_tensor = tf.convert_to_tensor(x_test_np)
x_test_tensor = tf.cast(x_test_tensor, tf.int64)

In [14]:
y_pred = model.predict(x_test_tensor)

트랜스포머
트랜스포머
81/81 [==============================] - 1s 10ms/step


In [15]:
yp=[]
for i in y_pred:
    yp.append(i.argmax())

## Temp 데이터 (train은 증강한거 test는 증강 안한거 8개 세션)

In [18]:
from sklearn import metrics
report = metrics.classification_report(y_test.to_list(), yp, output_dict=True, digits=5)
print(report['weighted avg'])

{'precision': 0.6619320817008985, 'recall': 0.42578885858979354, 'f1-score': 0.512278748482101, 'support': 2567}
